In [1]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np

Current working directory was changed to: C:\Users\Vinicius\Documents\GitHub\Previsor\App


In [2]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 29/12/22 - Horário: 11:38:16.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:41:20.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 29/12/22 - Horário: 11:42:20.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:35.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:43:48.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 29/12/22 - Horário: 11:44:27.  Portanto não foi baixado novamente.
vrapeelo

In [3]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]

In [4]:
monitoramentodados  = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))
io_vrapeel = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[['DthEnvio','IdeUsinaOutorga','DatPrevistaAprovacaoIII','DatRealizacaoIII']].drop_duplicates(inplace=True)

In [5]:
rename_cols = {'DthEnvio' : 'Data_Envio',
'NomUsina' : 'usina_nome',
'DatRealizacaoII' : 'canteiroReal',
'DatPrevistaAprovacaoIII' : 'prev_IO',
'DatRealizacaoIII' : 'IO_real',
'DatRealizacaoIX' : 'DesvRio_real',
'DatRealizacaoXI' : 'Ench_Real',
'DatRealizacaoXIII' : 'IOTrans_Real',
'DatRealizacaoXIV' : 'Conc_Trans_REAL',
'DatRealizacaoXII' : 'Comiss_Real',
'DatRealizacaoVII' : 'ME_Real_conc_eol',
'DatRealizacaoVIII' : 'Conc_Combust_Real',}
cols = ['IdeUsinaOutorga','DthEnvio','CodCeg','NomUsina','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV',]
vcronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [6]:
rename_cols = {
    'DatPrevistaComercial' : 'DatPrevistaComercial1'
}
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','SigTipoGeracao','DscObservacao','VlrPotenciaUnitaria','NumOperacaoUg','DatPrevistaComercial','DatPrevistaRotor','DatPrevistaMaximaRotor','DatRealizacaoRotor','DatPrevistaTeste','DatPrevistaMaximaTeste','DatRealizacaoTeste','DatPrevistaComercial','DatPrevistaMaximaComercial','DatRealizacaoComercial','SigTipoGeracao','NomUsina']
ugrapeel  = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [7]:
rename_cols = {
'DthEnvio' : 'Data_Envio',
'SigTipoGeracao' : 'TipoGeracao',
'NomUsina' : 'usina_nome',
'NumOperacaoUg' : 'Num_UG',
'VlrPotenciaUnitaria' : 'Pot_UG',
'DatRealizacaoRotor' : 'UG_decidarotor_real',
'DatPrevistaTeste' : 'UG_op_test_previsão',
'DatRealizacaoTeste' : 'UG_op_test_real',
'DatPrevistaComercial' : 'UG_op_com_previsão',
'DatRealizacaoComercial' : 'UG_op_com_real',
}

cols = ['DthEnvio','IdeUsinaOutorga','SigTipoGeracao','NomUsina','NumOperacaoUg','VlrPotenciaUnitaria','DatRealizacaoRotor','DatPrevistaTeste','DatRealizacaoTeste','DatPrevistaComercial','DatRealizacaoComercial','DscObservacao','NumOperacaoUg']
inputug = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [8]:
cols = ['IdeUsinaOutorga','NumUgUsina','MdaPotenciaUnitaria','MdaPotenciaLiberadaTeste','MdaPotenciaLiberadaComercial','NumDespachoTeste','numDespachoComercial','IdcMonitorada','DatDescidaoutorgadaRotor','DatRealizacaoSFGRotor','DatInicioOpTesteOutorgada','DatLiberacaoSFGTeste','DatUGInicioOpComerOutorgado','DatLiberOpComerRealizado','DatPrevisaoSFGComercial']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].drop_duplicates()

In [9]:
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','NomSituacaoI','PctEnergiaContratoAssinadoI','DatConclusaoI','NomSituacaoII','PctEnergiaContratoAssinadoII','DatConclusaoII','NomSituacaoIII','PctEnergiaContratoAssinadoIII','DatConclusaoIII','DscObservacao']
rename_cols = {'NomSituacaoI' : 'SituacaoACR',
'PctEnergiaContratoAssinadoI' : 'PctEnergiaContratoAssinadoACR',
'DatConclusaoI' : 'DatConclusaoACR',
'NomSituacaoII' : 'SituacaoCCESI',
'PctEnergiaContratoAssinadoII' : 'PctEnergiaContratoAssinadoCCESI',
'DatConclusaoII' : 'DatConclusaoCCESI',
'NomSituacaoIII' : 'SituacaoACL',
'PctEnergiaContratoAssinadoIII' : 'PctEnergiaContratoAssinadoACL',
'DatConclusaoIII' : 'DatConclusaoACL',
'DscObservacao' : 'DscObservacaomercado'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [10]:
cols= ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','DatSolicitacaoI','DatPrevistaEmissaoI','DatEmissaoI','DatValidadeI','NomSituacaoI','DatSolicitacaoIV','DatPrevistaEmissaoIV','DatEmissaoIV','DatValidadeIV','NomSituacaoIV','DatSolicitacaoV','DatPrevistaEmissaoV','DatEmissaoV','DatValidadeV','NomSituacaoV','IdcTipoEntrave','DscSolucao','DscObservacao']
rename_cols = {
'DatSolicitacaoI' : 'DatSolicitacaoLP',
'DatPrevistaEmissaoI' : 'DatPrevistaEmissaoLP',
'DatEmissaoI' : 'DatEmissaoLP',
'DatValidadeI' : 'DatValidadeLP',
'NomSituacaoI' : 'NomSituacaoLP',
'DatSolicitacaoIV' : 'DatSolicitacaoLI',
'DatPrevistaEmissaoIV' : 'DatPrevistaEmissaoLI',
'DatEmissaoIV' : 'DatEmissaoLI',
'DatValidadeIV' : 'DatValidadeLI',
'NomSituacaoIV' : 'NomSituacaoLI',
'DatSolicitacaoV' : 'DatSolicitacaoLO',
'DatPrevistaEmissaoV' : 'DatPrevistaEmissaoLO',
'DatEmissaoV' : 'DatEmissaoLO',
'DatValidadeV' : 'DatValidadeLO',
'NomSituacaoV' : 'NomSituacaoLO',
'DscObservacao' : 'DscObservacaoAmbiental'}
ambientaldatas =  pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [11]:
cols = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoI','DatAssinaturaContratoI','DatValidadeContratoI','NomSitContratoII','DatAssinaturaContratoII','DatValidadeContratoII','NomSituacaoContratoIII','DatAssinaturaContratoIII','DatValidadeContratoIII','NomSitContratoIV','DatAssinaturaContratoIV','DatValidadeContratoIV','DscObservacao']
rename_cols = {'NomSitContratoI' : 'NomSitContratoCCD',
'DatAssinaturaContratoI' : 'DatAssinaturaContratoCCD',
'DatValidadeContratoI' : 'DatValidadeContratoCCD',
'NomSitContratoII' : 'NomSitContratoCCT',
'DatAssinaturaContratoII' : 'DatAssinaturaContratoCCT',
'DatValidadeContratoII' : 'DatValidadeContratoCCT',
'NomSituacaoContratoIII' : 'NomSituacaoContratoCUSD',
'DatAssinaturaContratoIII' : 'DatAssinaturaContratoCUSD',
'DatValidadeContratoIII' : 'DatValidadeContratoCUSD',
'NomSitContratoIV' : 'NomSitContratoCUST',
'DatAssinaturaContratoIV' : 'DatAssinaturaContratoCUST',
'DatValidadeContratoIV' : 'DatValidadeContratoCUST',
'DscObservacao' : 'DscObservacaoAcesso'}
acessocontratos =  pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].drop_duplicates().rename(columns=rename_cols)

In [12]:
cols1 = ['IdeUsinaOutorga','DthEnvio','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','SituacaoACR','PctEnergiaContratoAssinadoACR','DatConclusaoACR','SituacaoCCESI','PctEnergiaContratoAssinadoCCESI','DatConclusaoCCESI','SituacaoACL','PctEnergiaContratoAssinadoACL','DatConclusaoACL','DscObservacaomercado']
cols2 = ['IdeUsinaOutorga','DthEnvio','DatSolicitacaoLP','DatPrevistaEmissaoLP','DatEmissaoLP','DatValidadeLP','NomSituacaoLP','DatSolicitacaoLI','DatPrevistaEmissaoLI','DatEmissaoLI','DatValidadeLI','NomSituacaoLI','DatSolicitacaoLO','DatPrevistaEmissaoLO','DatEmissaoLO','DatValidadeLO','NomSituacaoLO','IdcTipoEntrave','DscSolucao','DscObservacaoAmbiental']
cols3 = ["IdeUsinaOutorga","DthEnvio",'IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoCCD','DatAssinaturaContratoCCD','DatValidadeContratoCCD','NomSitContratoCCT','DatAssinaturaContratoCCT','DatValidadeContratoCCT','NomSituacaoContratoCUSD','DatAssinaturaContratoCUSD','DatValidadeContratoCUSD','NomSitContratoCUST','DatAssinaturaContratoCUST','DatValidadeContratoCUST','DscObservacaoAcesso', ]
inputvrapeelv = pd.merge(mercado[cols1],ambientaldatas[cols2],on=["IdeUsinaOutorga","DthEnvio"],how="inner")
inputvrapeelv = pd.merge(inputvrapeelv,acessocontratos[cols3],on=["IdeUsinaOutorga","DthEnvio"],how="inner")

In [13]:
rapeelatualizado = inputvrapeelv.loc[:,['IdeUsinaOutorga','DthEnvio']]
rapeelatualizado =rapeelatualizado.loc[rapeelatualizado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()].rename(columns={'DthEnvio': 'DthEnvioMAX'})

In [14]:
cols1 = ['DthEnvio','IdeUsinaOutorga','CodCeg','NomUsina','PctAoACR','PctAoACL','PctAutoproducao','SituacaoACR','PctEnergiaContratoAssinadoACR','DatConclusaoACR','SituacaoCCESI','PctEnergiaContratoAssinadoCCESI','DatConclusaoCCESI','SituacaoACL','PctEnergiaContratoAssinadoACL','DatConclusaoACL','IdcTipoInstalacao','IdcNecessidade','NomSituacaoParAcesso','DatEmissaoParAcesso','NomSitContratoCCD','DatAssinaturaContratoCCD','DatValidadeContratoCCD','NomSitContratoCCT','DatAssinaturaContratoCCT','DatValidadeContratoCCT','NomSituacaoContratoCUSD','DatAssinaturaContratoCUSD','DatValidadeContratoCUSD','NomSitContratoCUST','DatAssinaturaContratoCUST','DatValidadeContratoCUST','DatSolicitacaoLP','DatPrevistaEmissaoLP','DatEmissaoLP','DatValidadeLP','NomSituacaoLP','DatSolicitacaoLI','DatPrevistaEmissaoLI','DatEmissaoLI','DatValidadeLI','NomSituacaoLI','DatSolicitacaoLO','DatPrevistaEmissaoLO','DatEmissaoLO','DatValidadeLO','NomSituacaoLO','IdcTipoEntrave','DscSolucao','DscObservacaoAcesso','DscObservacaomercado','DscObservacaoAmbiental']
cols2 = ['IdeUsinaOutorga','DthEnvioMAX']
cols3 = ['IdeUsinaOutorga','DscObservacao','DthEnvio']
definicoes = pd.merge(inputvrapeelv[cols1],rapeelatualizado[cols2],left_on=['IdeUsinaOutorga','DthEnvio'],right_on=['IdeUsinaOutorga','DthEnvioMAX'],how="inner")
definicoes = pd.merge(definicoes,ugrapeel[cols3],on=['IdeUsinaOutorga','DthEnvio'],how="inner").drop_duplicates().rename(columns={
    'DscObservacao' : 'DscObservacaoUG'
})

dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

definicoes['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    definicoes.loc[definicoes.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = definicoes[validade]

In [15]:
definicoesmonitoramento = pd.merge(monitoramentodados,definicoes,on="IdeUsinaOutorga",how="left")

definicoesmonitoramento['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    definicoesmonitoramento.loc[definicoesmonitoramento[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]
definicoesmonitoramento.loc[definicoesmonitoramento.CondicaoAmbiental.isna(),'CondicaoAmbiental'] = 'Sem LP'
definicoesmonitoramento[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental,CondicaoAmbiental
1178,NaT,2024-08-12,2021-04-01,2024-08-12,LI
852,2027-11-22,2024-01-23,2019-09-13,2027-11-22,LO
471,NaT,2026-04-29,2022-06-30,2026-04-29,LI
2555,NaT,NaT,NaT,NaT,Sem LP
1009,2027-08-22,2025-12-22,2021-05-11,2027-08-22,LO
902,NaT,2021-06-09,2018-11-01,2021-06-09,LI
1189,2032-03-03,2032-03-03,2032-03-03,2032-03-03,LO
317,2032-02-03,2032-02-03,2032-02-03,2032-02-03,LO
2388,NaT,NaT,NaT,NaT,Sem LP
2217,NaT,NaT,2026-03-10,2026-03-10,LP


In [16]:
definicoesmonitoramento['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSitContratoCCD ==  nsa)&
    (definicoesmonitoramento.NomSitContratoCCT == nsa) &
    definicoesmonitoramento.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

definicoesmonitoramento.loc[
(    ((definicoesmonitoramento.NomSitContratoCCD ==  valido) |
    (definicoesmonitoramento.NomSitContratoCCT == valido)) &
    definicoesmonitoramento.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.NomSitContratoCCD ==  vencido) |
    (definicoesmonitoramento.NomSitContratoCCT == vencido) |
    (definicoesmonitoramento.NomSitContratoCCD == nass) |
    (definicoesmonitoramento.NomSitContratoCCT == nass )) &
    definicoesmonitoramento.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(definicoesmonitoramento[['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


definicoesmonitoramento['CondicaoUso'] = "Não informado"

definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSituacaoContratoCUSD ==  nsa)&
    (definicoesmonitoramento.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


definicoesmonitoramento.loc[
    ((definicoesmonitoramento.NomSituacaoContratoCUSD ==  valido) |
    (definicoesmonitoramento.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.NomSituacaoContratoCUSD ==  vencido) |
    (definicoesmonitoramento.NomSitContratoCUST == vencido) |
    (definicoesmonitoramento.NomSituacaoContratoCUSD == nass) |
    (definicoesmonitoramento.NomSitContratoCUST == nass )) &
    definicoesmonitoramento.DthEnvio.notna())
    ,'CondicaoUso'] = "Sem Uso"

display(definicoesmonitoramento[['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


definicoesmonitoramento["PPA"] = "Ambos"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "ACR") & 
        definicoesmonitoramento.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoesmonitoramento.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

definicoesmonitoramento.loc[
    (((definicoesmonitoramento.DscComercializacaoEnergia == "Fora do ACR") & 
        definicoesmonitoramento.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(definicoesmonitoramento[['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
2726,NaN,NaN,NaT,Não informado
1380,Não se Aplica,Válido,2022-12-05 14:56:02.657,OK
2314,Não se Aplica,Não Assinado,2022-12-01 09:06:45.470,Sem Conexão
2796,Não se Aplica,Não Assinado,2022-12-04 17:46:19.890,Sem Conexão
2716,Não se Aplica,Não Assinado,2022-12-05 10:03:54.997,Sem Conexão
2655,Não se Aplica,Não Assinado,2022-12-05 16:31:52.323,Sem Conexão
537,Válido,Não se Aplica,2022-12-05 19:43:12.560,OK
466,Não se Aplica,Válido,2021-09-02 15:43:52.157,OK
1029,Não se Aplica,Não Assinado,2022-12-05 08:45:52.020,Sem Conexão
1235,Não se Aplica,Válido,2022-12-05 00:06:01.703,OK


,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
1865,Não se Aplica,Não Assinado,Sem Conexão
1532,Não se Aplica,Não Assinado,Sem Conexão
1596,Não se Aplica,Válido,OK
1756,Não se Aplica,Válido,OK
1573,Não se Aplica,Não Assinado,Sem Conexão
958,Não se Aplica,Não Assinado,Sem Conexão
1667,Não se Aplica,Não Assinado,Sem Conexão
2642,Não se Aplica,Não Assinado,Sem Conexão
447,Não se Aplica,Válido,OK
2579,NaN,NaN,Não informado


,DscComercializacaoEnergia,DatConclusaoACL,PPA
2586,Fora do ACR,NaT,Nenhum
2428,Fora do ACR,NaT,Nenhum
1809,Fora do ACR,NaT,Nenhum
1585,Fora do ACR,NaT,Nenhum
779,Fora do ACR,2021-05-03,ACL
2538,ACR,NaT,ACR
698,Fora do ACR,NaT,Nenhum
98,Fora do ACR,NaT,Nenhum
40,Fora do ACR,NaT,Nenhum
1518,Fora do ACR,NaT,Nenhum


In [17]:
tabelcriterio = definicoesmonitoramento.loc[definicoesmonitoramento.IdcUsinaMonitorada == "Sim"].copy()

In [18]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

tabelcriterio['criterio'] = np.select(
    [
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


tabelcriterio['dscriterio'] = np.select(
    [
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(tabelcriterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
1932,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2990,Não Iniciada,Nenhum,Sem LP,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
1307,Em andamento,Nenhum,LI,OK,0,Usina em Obras em andamento
831,Em andamento,Nenhum,LO,OK,0,Usina em Obras em andamento
2160,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1517,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
2623,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
67,Em andamento,Nenhum,LI,Sem Uso,0,Usina em Obras em andamento
418,Não Iniciada,Nenhum,LI,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2425,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"


In [19]:
slp = "Sem LP"
sim = "Sim"
tabelcriterio['criterio_novo'] = np.select(
    [
        (tabelcriterio.IdcSemPrevisao == sim),
        (tabelcriterio.IdcSituacaoObra == ea),
        (( tabelcriterio.IdcSituacaoObra == na)  &  (tabelcriterio.PPA != nenhum) ),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso == ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental.isin(["LI","LO"])) & (tabelcriterio.CondicaoUso != ok)),
        ((tabelcriterio.IdcSituacaoObra == na) & (tabelcriterio.PPA == nenhum) & (tabelcriterio.CondicaoAmbiental == "LP") & (tabelcriterio.CondicaoUso != ok)),
        (tabelcriterio.IdcSituacaoObra == paralisada),
        (tabelcriterio.CondicaoAmbiental == "Sem LP"),
        ((tabelcriterio.DthEnvio.isna()) & (tabelcriterio.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(tabelcriterio[['IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(10))

,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
2772,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1367,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
2957,Não Iniciada,Nenhum,Sem LP,Não informado,6,"Sem obras, sem PPA e sem Rapeel"
1942,Não Iniciada,Nenhum,LP,Sem Uso,9,"Sem obras, sem PPA e LP OK e Cust não Ok"
29,Não Iniciada,ACR,LI,OK,9,Sem obras e PPA Ok
2674,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
829,Não Iniciada,Nenhum,LP,Sem Uso,9,"Sem obras, sem PPA e LP OK e Cust não Ok"
1746,Não Iniciada,Nenhum,LO,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
2514,Não Iniciada,Nenhum,Sem LP,Não informado,6,"Sem obras, sem PPA e sem Rapeel"
2950,Não Iniciada,Nenhum,Sem LP,Não informado,6,"Sem obras, sem PPA e sem Rapeel"


In [20]:
direcory_sas_files = r"C:\Users\viniciusbarros\OneDrive - unb.br\0Aneel\Previsor"

In [21]:
cols1 = ['Data_Envio','IdeUsinaOutorga','CodCeg','usina_nome','canteiroReal','prev_IO','IO_real','DesvRio_real','Ench_Real','IOTrans_Real','Conc_Trans_REAL','Comiss_Real','ME_Real_conc_eol','Conc_Combust_Real','DatRealizacaoIV','DatRealizacaoV','DatRealizacaoVI','DatRealizacaoX']
vinputcronograma = vcronograma[cols1].drop_duplicates()
vinputcronograma["classe"] = vinputcronograma.CodCeg.str.slice(3,5)
vinputcronograma["CC_real"] = pd.to_datetime(pd.NA)
vinputcronograma.loc[vinputcronograma.classe == "PH","CC_real"] = vinputcronograma.DatRealizacaoIV
vinputcronograma.loc[vinputcronograma.classe == "CV","CC_real"] = vinputcronograma.DatRealizacaoV

vinputcronograma["ME_real"] = vinputcronograma.DatRealizacaoX
vinputcronograma.loc[vinputcronograma.classe == "CV","ME_real"] = vinputcronograma.DatRealizacaoVI

In [22]:
cols = ['IdeUsinaOutorga','CodCeg','Data_Envio','usina_nome','canteiroReal','prev_IO','IO_real','CC_real','DesvRio_real','Ench_Real','IOTrans_Real','Conc_Trans_REAL','Comiss_Real','ME_real','ME_Real_conc_eol','Conc_Combust_Real']
rename_cols = {
    #'IdeUsinaOutorga' : 'Idempreendimento'
}
datasreal = vinputcronograma[cols].drop_duplicates().rename(columns=rename_cols)


cols1 = ['IdeUsinaOutorga','Data_Envio','usina_nome','prev_IO','canteiroReal','IO_real','CC_real','DesvRio_real','ME_real','ME_Real_conc_eol','Ench_Real','Conc_Combust_Real','Comiss_Real','IOTrans_Real','Conc_Trans_REAL',]
cols2 = ['IdeUsinaOutorga','Data_Envio', 'DscObservacao',  'UG_op_com_previsão']
cols3 = ['IdeUsinaOutorga','DthEnvioMAX']
aux = pd.merge(datasreal[cols1],
        inputug[cols2],how= 'inner', on=['IdeUsinaOutorga','Data_Envio'])
realcronogramarapeel = pd.merge(aux,rapeelatualizado,left_on=['IdeUsinaOutorga','Data_Envio'],right_on=['IdeUsinaOutorga','DthEnvioMAX'],how='inner')
rename_cols = {
    
}